In [1]:
import numpy as np
import pandas as pd

목표 : $ \ P(Y|X_1,X_2,X_3...X_p) $의 maximize
#### $ \ P(Y|X_1,X_2,X_3...X_p) = \frac{{P(X_1,X_2,...X_p|Y) \cdot P(Y) }}{{P(X_1,X_2,...X_p)}} \ $

분모는 이미 알고있는 상수, 목표는 분자의 maximize

${P(X_1,X_2,...X_p|Y) \cdot P(Y)}$

chain rule 사용 -> 
$P(X_1,X_2,...X_p|Y)= \frac {P(X_1,X_2,...X_p,Y)}{P(X_2,X_3...X_p,Y)} \cdot \frac {P(X_2,X_3...X_p,Y)}{P(Y)} \cdot P(Y)=
P(X_1|X_2,X_3..X_p,Y) \cdot P(X_2,X_3...X_p|Y) \cdot P(Y)
$

위 과정을 계속 하면 다음과 같은 식을 얻음

${P(X_1|X_2,...X_p,Y) \cdot P(X_2|X_3,...X_p,Y) \cdot P(X_3|X_4,...X_p,Y)  \cdot P(y)}$

Conditional Independence(조건부 독립?)

X,Y,Z: random variables, P(X|Y,Z)=P(X|Z)

따라서 ${P(X_1,X_2,...X_p|Y) \cdot P(Y)} =P(X_1|Y) \cdot P(X_2|Y) \cdot ... P(X_p|Y) \cdot P(Y) $

문제점: 확률변수의 곱이므로, 하나라도 0이라면? 0으로 간주됨.

해결방법: Laplace or m-estimate ->  공통으로 0이 되는 것을 막아줌

In [15]:
HomeOwner=['Y','N','N','Y','N','N','Y','N','N','N']
M=['Single','Married','Single','Married','Divorced','Married','Divorced','Single','Married','Single']
A=['125k','100k','70k','120k','95k','60k','220k','85k','75k','90k']
D=['N','N','N','N','Y','N','N','Y','N','Y']
data=pd.DataFrame({'Homeowner':HomeOwner,
                   'Marital Status':M,
                   'Annual Income': A,
                   'Defaulted Borrower': D})

In [16]:
data.head()

,Homeowner,Marital Status,Annual Income,Defaulted Borrower
0,Y,Single,125k,N
1,N,Married,100k,N
2,N,Single,70k,N
3,Y,Married,120k,N
4,N,Divorced,95k,Y


Continuous variable일때( 위의 income의 경우)

1. bin으로 나누기 -> 독립성 가정에 대한 위반을 할 수 있음
2. two-way split -> 정보 손실이 많음
3. normal distribution으로 가정

test record: X=(Home owner: Yes,Married,Income=120k) Deafaulted Borrower인가? Y/N

P(X|DB=No) vs P(X|DB=Yes) 중 더 높은 확률로 예측

1. P(Home own: No | DB=No) x P(Married | DB=No) x P(Income=120k | DB=No)
2. P(Home own: No | DB=Yes) x P(Married | DB=Yes) x P(Income=120k | DB=Yes)

In [67]:
# P(income=120k| DB= Yes or No) 구하기
from scipy.stats import norm

data['Annual Income']=data['Annual Income'].str.replace('k','')

# DB=No 인 사람들의 경우

DB_no=data[data['Defaulted Borrower']=='N']['Annual Income']
DB_no=DB_no.astype(int)
no_mean=np.mean(DB_no)
no_var=np.var(DB_no)

dist = norm(loc=no_mean, scale=np.sqrt(no_var))
prob_no= dist.pdf(120)

# DB=yes인 사람들의 경우

DB_yes=data[data['Defaulted Borrower']=='Y']['Annual Income']
DB_yes=DB_yes.astype(int)
yes_mean=np.mean(DB_yes)
yes_var=np.var(DB_yes)

dist = norm(loc=yes_mean, scale=np.sqrt(yes_var))
prob_yes= dist.pdf(120)


1.8366850019098434e-13

In [69]:
# P(X|DB=No)
4/7 * 4/7 * prob_no

0.0025295793085132836

In [70]:
# P(X|DB=Yes), 0이 있기에 0으로 간주됨. -> Laplace사용
1 * 0 * prob_yes

0.0